In [10]:
import aiohttp
import asyncio
import requests 
import pandas as pd 

In [11]:


async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

async def retry(session, url, max_retries=3):
    for attempt in range(max_retries + 1):
        try:
            return await fetch_data(session, url)
        except Exception as e:
            if attempt < max_retries:
                delay = 2 ** attempt
                await asyncio.sleep(delay/10)
            else:
                return await retry(session, url)

async def get_data_from_API(city):
    res = []
    tasks = []
    
    historico_alerta_api = "https://api.mosqlimate.org/api/datastore/historico_alerta/"
    filters = "disease=%s&start=%s&end=%s&geocode=%i" % ("dengue", "2010-01-01", "2023-12-30", city)

    
    pagination = f"?page=1&per_page=100&"
    url = historico_alerta_api + pagination + filters
    
    resp = requests.get(historico_alerta_api + pagination + filters)
    last_page = resp.json()["pagination"]['total_pages']
    
    resp = resp.json()
    
    #tasks.append(resp)
    
    res.extend(resp['items'])

    async with aiohttp.ClientSession() as session:
        for i in range(2, last_page+1):
            pagination = f"?page={i}&per_page=100&"
            url = historico_alerta_api + pagination + filters
            data = retry(session, url)
            tasks.append(data)

        results = await asyncio.gather(*tasks)

    for result in results:
        res.extend(result["items"])

    return res

In [12]:
res = await get_data(2304400)

In [13]:
res

[{'data_iniSE': '2023-07-23',
  'SE': 202330,
  'casos_est': 342.5,
  'casos_est_min': 70,
  'casos_est_max': None,
  'casos': 20,
  'municipio_geocodigo': 2304400,
  'p_rt1': 0.5414351,
  'p_inc100k': 12.748399,
  'Localidade_id': 0,
  'nivel': 1,
  'id': 230440020233019571,
  'versao_modelo': '2023-08-02',
  'Rt': 1.0079938,
  'municipio_nome': 'Fortaleza',
  'pop': 2686612.0,
  'tempmin': 26.5714285714286,
  'umidmax': 81.5307345714286,
  'receptivo': 0,
  'transmissao': 0,
  'nivel_inc': 1,
  'umidmed': 80.2562521428571,
  'umidmin': 78.8008808571429,
  'tempmed': 26.7448978571429,
  'tempmax': 26.8571428571429,
  'casprov': None,
  'casprov_est': None,
  'casprov_est_min': None,
  'casprov_est_max': None,
  'casconf': None},
 {'data_iniSE': '2023-07-16',
  'SE': 202329,
  'casos_est': 331.0,
  'casos_est_min': 168,
  'casos_est_max': None,
  'casos': 127,
  'municipio_geocodigo': 2304400,
  'p_rt1': 0.5610516,
  'p_inc100k': 12.32035,
  'Localidade_id': 0,
  'nivel': 1,
  'id': 23

In [9]:
df = pd.DataFrame(res)

df.set_index('data_iniSE', inplace = True)
    
df.index = pd.to_datetime(df.index)

df.sort_index()

,SE,casos_est,casos_est_min,casos_est_max,casos,municipio_geocodigo,p_rt1,p_inc100k,Localidade_id,nivel,...,nivel_inc,umidmed,umidmin,tempmed,tempmax,casprov,casprov_est,casprov_est_min,casprov_est_max,casconf
data_iniSE,,,,,,,,,,,,,,,,,,,,,
2010-01-03,201001,56.0,56,56.0,56,2304400,0.000000,2.084410,0,1,...,0,NaN,NaN,NaN,NaN,None,None,None,None,None
2010-01-10,201002,55.0,55,55.0,55,2304400,0.000000,2.047188,0,1,...,0,NaN,NaN,NaN,NaN,None,None,None,None,None
2010-01-17,201003,77.0,77,77.0,77,2304400,0.000000,2.866063,0,2,...,0,NaN,NaN,NaN,NaN,None,None,None,None,None
2010-01-24,201004,56.0,56,56.0,56,2304400,0.000000,2.084410,0,2,...,0,NaN,NaN,NaN,NaN,None,None,None,None,None
2010-01-31,201005,114.0,114,114.0,114,2304400,0.000000,4.243262,0,2,...,0,NaN,NaN,NaN,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-25,202326,293.0,243,NaN,230,2304400,0.178709,10.905930,0,1,...,1,81.729607,80.055820,26.500000,26.736842,None,None,None,None,None
2023-07-02,202327,330.0,257,NaN,236,2304400,0.752130,12.283129,0,1,...,1,83.078331,79.561093,26.675000,27.000000,None,None,None,None,None
2023-07-09,202328,361.0,259,NaN,232,2304400,0.964395,13.436998,0,1,...,1,79.305869,76.619321,26.924812,27.000000,None,None,None,None,None
